In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid3_iu_27000_1/"

"../../graphs/grid3_iu_27000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.728843 seconds (535.44 k allocations: 29.664 MB, 0.68% gc time)
  0.330518 seconds (539.67 k allocations: 12.913 MB, 1.79% gc time)
  8.866985 seconds (86.76 M allocations: 2.193 GB, 4.29% gc time)
Working on iteration 10 with a-norm error 27.10654578009891
Working on iteration 20 with a-norm error 22.161120310295626
Working on iteration 30 with a-norm error 17.97072386910171
Working on iteration 40 with a-norm error 15.015583146455878
Working on iteration 50 with a-norm error 12.430769534493532
Working on iteration 60 with a-norm error 10.368138687506136
Working on iteration 70 with a-norm error 8.559805068662799
Working on iteration 80 with a-norm error 7.296184430811318
Working on iteration 90 with a-norm error 5.989771453404592
Working on iteration 100 with a-norm error 4.957643091291368
Working on iteration 110 with a-norm error 4.127660737833292
Working on iteration 120 with a-norm error 3.5155133249370363
Working on iteration 130 with a-norm error 2.8279586492627278
Working 